# FATES_INCLINE_dataprep_surfacedata

Heavily inspired from NorESM-LSP notebooks and https://github.com/huitang-earth/MossLichen_testbed/blob/main/scripts/SeedClim_surfacedata_modification.ipynb 

Script by Eva Lieungh, Elin CR Aas, Hui Tang
Started 2022-11-15

### Data
Observational data from the Vestland Climate Grid  stored on OSF: 
https://osf.io/npfa9

VCG data files:
- https://osf.io/s9k7c, VCG_clean_gridded_daily_climate_2008-2022.csv
- VCG_clean_soil_chemistry_2009_2010_2013_2015.csv
- VCG_clean_soil_structure_2013_2014_2018.csv
- VCG_clean_soilmoisture_plotlevel_2015-2018.csv


### This script

... reads observational data, downloads LSP input data, runs a loop to modify the surface data, and re-uploads the new input data to a repository. 

In [1]:
# import libraries
import xarray as xr  # NetCDF data handling
import netCDF4 
import matplotlib.pyplot as plt  # Plotting
import time  # Keeping track of runtime
import json  # For reading data dictionaries stored in json format
import pandas as pd  # Tabular data analysis
import datetime as dt  # For workaround with long simulations (beyond year 2262)
import statistics as stats # For mean and other calculations
from pathlib import Path  # For easy path handling
import zipfile # for unzipping
import shutil # easiest whole-directory zipping
import glob # for wildcard * searching in file names

In [2]:
# set path observational data
observations_path = Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data/OSF_VCG")

# set path to default surface data
surfacedata_path = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data/LSPv1_default_inputfiles"))

# set path for where to store modified surface data
modified_surfacedata_path = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata"))

In [3]:
# define LSP site identities and corresponding names 
siteID = ["ALP1","ALP2","ALP3","ALP4","SUB1","SUB2","SUB3","SUB4","BOR1","BOR2","BOR3","BOR4"]
siteID1 = ["Ulvehaugen","Lavisdalen","Gudmedalen","Skjelingahaugen",
           "Alrust","Hogsete","Rambera","Veskre",
           "Fauske","Vikesland","Arhelleren","Ovstedalen"] 
siteID2 = ["Ulvehaugen","Lavisdalen","Gudmedalen","Skjellingahaugen",
           "Alrust","Hogsete","Rambera","Veskre",
           "Fauske","Vikesland","Arhelleren","Ovstedalen"]
siteID3 = ["ULV","LAV","GUD","SKJ","ALR","HOG","RAM","VES","FAU","VIK","ARH","OVS"]

## Download and unzip default surface data

It is stored here: https://github.com/NorESMhub/noresm-lsp-data/tree/main/sites. Change directories to where the data should be stored, and then download from URLs with wget into site-specific folders. 

In [4]:
%%bash
cd /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/3_FATES_INCLINE/data/LSPv1_default_inputfiles
wget -P ALP1 https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP1.zip
wget -P ALP2 https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP2.zip
wget -P ALP3 https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP3.zip
wget -P ALP4 https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP4.zip

--2023-02-23 15:01:33--  https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39223589 (37M) [application/zip]
Saving to: ‘ALP1/ALP1.zip.1’

     0K .......... .......... .......... .......... ..........  0% 3.61M 10s
    50K .......... .......... .......... .......... ..........  0% 7.71M 8s
   100K .......... .......... .......... .......... ..........  0% 6.95M 7s
   150K .......... .......... .......... .......... ..........  0% 10.2M 6s
   200K .......... .......... .......... .......... ..........  0% 6.39M 6s
   250K .......... .......... .......... .......... ..........  0% 4.38M 6s
   300K .......... .......... .......... .......... ..........  0% 12.8M 6s
   350K .......... 

Unzip the files into the modified_surfacedata_path. These copies will be modified further down.

In [5]:
for i in range(0,1):
    with zipfile.ZipFile(surfacedata_path + "/" + siteID[i] + "/" + siteID[i] + ".zip", 'r') as zip_ref:
          zip_ref.extractall(modified_surfacedata_path + "/" + siteID[i])

## Read in the observational data

and print relevant info

In [6]:
soil_moisture = pd.read_csv(observations_path / "VCG_clean_soilmoisture_plotlevel_2015-2018.csv",
                            index_col=None)
soil_moisture.head()

,year,date,siteID,turfID,blockID,blockID_FC,replicate,value,weather,recorder,comments,transcriber_comment
0,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,1,28.7,cloudy,siri,NaN,NaN
1,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,2,30.8,cloudy,siri,NaN,NaN
2,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,3,35.3,cloudy,siri,NaN,NaN
3,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,4,26.7,cloudy,siri,NaN,NaN
4,2015.0,2015-06-03,Alrust,7TT233,Alr2,2,1,34.5,cloudy,siri,NaN,NaN


In [7]:
soil_chemistry = pd.read_csv(observations_path / "VCG_clean_soil_chemistry_2009_2010_2013_2015.csv",
                             index_col=None)
print("available variables: ", soil_chemistry.variable.unique())
print("site names: ", soil_chemistry.siteID_dest.unique()) # matches siteID1 list

available variables:  ['pH' 'loi' 'NO3N' 'NH4N' 'available_N' 'soil_depth' 'N_content'
 'C_content' 'CN_ratio']
site names:  ['Arhelleren' 'Fauske' 'Gudmedalen' 'Hogsete' 'Lavisdalen' 'Rambera'
 'Skjelingahaugen' 'Ulvehaugen' 'Veskre' 'Vikesland' 'Ovstedalen' 'Alrust']


In [8]:
soil_structure = pd.read_csv(observations_path / "VCG_clean_soil_structure_2013_2014_2018.csv", 
                             index_col=None)
print("available variables: ", soil_structure.variable.unique())
print("site names: ", soil_structure.siteID.unique()) # matches siteID1

# for some reason, only the alpine sites have data for clay, silt and sand 
print("sites with clay/silt/sand observations: ", soil_structure[soil_structure["variable"] == "sand"].siteID.unique())

available variables:  ['soil_depth' 'clay' 'silt' 'sand' 'bulk_density']
site names:  ['Arhelleren' 'Alrust' 'Fauske' 'Gudmedalen' 'Hogsete' 'Lavisdalen'
 'Ovstedalen' 'Rambera' 'Skjelingahaugen' 'Ulvehaugen' 'Veskre'
 'Vikesland' 'ARH']
sites with clay/silt/sand observations:  ['Ulvehaugen' 'Lavisdalen' 'Gudmedalen' 'Skjelingahaugen']


In [9]:
INCLINE_metadata = pd.read_csv("C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data/INCLINE/INCLINE_metadata.csv", sep=";", index_col=None)
INCLINE_metadata.head()

,turfID,plotID,siteID,blockID,plot,OTC,treatment,precipitation_1960-1990,precipitation_2009-2019,temperature_1960-1990,temperature_2009-2019,coordinate_N,coordinate_E,elevation,slope,aspect
0,Gud_1_1,Gud_1_1,Gudmedalen,1,1,W,R,1925,2130,"5,87","6,04",6049880,710301,1117,16,S/SW
1,Gud_1_2,Gud_1_2,Gudmedalen,1,2,W,N,1925,2130,"5,87","6,04",6049880,710301,1117,17,S/SW
2,Gud_1_3,Gud_1_3,Gudmedalen,1,3,W,C,1925,2130,"5,87","6,04",6049880,710301,1117,29,S/SW
3,Gud_1_4,Gud_1_4,Gudmedalen,1,4,C,E,1925,2130,"5,87","6,04",6049879,710301,1117,17,S/SW
4,Gud_1_5,Gud_1_5,Gudmedalen,1,5,C,C,1925,2130,"5,87","6,04",6049879,710302,1117,14,S/SW


## In one big loop, for each site, 

1. subset the variables from observation data sets
2. calculate means of the variables,
3. replace the default in the surfacedata file, 
4. and save the file in the modified_surfacedata_path

Subset carbon content, calculate site mean. Needs to be combined with bulk density to convert to kg/m3

From CLM tech note about soil:
"The soil texture and organic matter content determine soil thermal and hydrologic properties (sections 6.3 and 7.4.1). The International Geosphere-Biosphere Programme (IGBP) soil dataset (Global Soil Data Task 2000) of 4931 soil mapping units and their sand and clay content for each soil layer were used to create a mineral soil texture dataset (Bonan et al. 2002b). Soil organic matter data is merged from two sources. The majority of the globe is from ISRIC-WISE (Batjes, 2006). The high latitudes come from the 0.25o version of the Northern Circumpolar Soil Carbon Database (Hugelius et al. 2012). Both datasets report carbon down to 1m depth. Carbon is partitioned across the top seven CLM4 layers (\sim1m depth) as in Lawrence and Slater (2008)."
https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Ecosystem/CLM50_Tech_Note_Ecosystem.html#surface-characterization

#### more variables that can be added to the loop below:

- plant cover estimates and bare ground estimates: can be calculated from subplot-cover of "bare"+"rock" from INCLINE community data, and on plot level from SeedClim data

In [10]:
for i in range(0,1):
    print("-------------------------------------")
    print("site: ", siteID[i], siteID2[i])

    #------------- GET OBSERVATION DATA -------------# 

    # Subset variables, calculate means, do unit conversions

    # Organic matter #
    carbon_observed = soil_chemistry[(soil_chemistry["siteID_dest"]==siteID1[i])
                                     & (soil_chemistry["variable"] == "C_content")]
    soil_bulk_density = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "bulk_density")]
    carbon_mean = stats.mean(carbon_observed["value"])
    soil_bulk_density_mean = stats.mean(soil_bulk_density["value"])
    print("mean carbon content (%): ", carbon_mean)
    print("bulk density (g/cm^3):   ", soil_bulk_density_mean)
        # observed bulk density: g/cm3, observed carbon content: %, model needs organic: kg/m3. 
        # To get total organic matter (not just C), divide by 0.58   
    org_obs = (soil_bulk_density_mean*1000)*(carbon_mean/100)/0.58 
    org_obs = min(110, org_obs)      # model assumes less than 130 kg/m3 organic matter 
    print("organic matter for model (kg/m3): ", org_obs)

    # percent sand # (only for ALP1-4)
    sand_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "sand")]
    sand_obs_mean = stats.mean(sand_observed["value"]) * 100
    print("sand %: ", sand_obs_mean)

    ## silt #
    #silt_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
    #                                   & (soil_structure["variable"] == "silt")]
    #silt_obs_mean = stats.mean(silt_observed["value"]) * 100
    #print("silt %: ", silt_obs_mean)

    # clay #
    clay_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "clay")]
    clay_obs_mean = stats.mean(clay_observed["value"]) * 100
    print("clay %: ", clay_obs_mean)

    # Soil depth#
    soil_depth = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "soil_depth")]
    soil_depth_mean = stats.mean(soil_depth["value"]) / 100 # convert cm to m
    print("soil depth, m: ", soil_depth_mean)

    # Slope # (NB! from INCLINE data - not identical to SeedClim and FunCaB data)
    slope_observed = INCLINE_metadata[INCLINE_metadata["siteID"]==siteID2[i]]
    slope_obs_mean = stats.mean(slope_observed["slope"])
    print("slope: ",slope_obs_mean)

    # plant and bare ground cover # - set to a fixed number for now! To be calculated from data
    plant_cover_obs = 90 # % cover

    # plant height # 
    plant_height_obs = 15/100
    #MONTHLY_HEIGHT_TOP

    #------------- MODIFY SURFACE DATA VARIABLES -------------#

    # open site-specific default surface data file to be modified
    file_pattern = modified_surfacedata_path + '/' + siteID[i] + '/surfdata*.nc'
    file_list = glob.glob(file_pattern)

    # check if at least one file was found
    if len(file_list) == 0:
        print(f"No file found matching the pattern {file_pattern}")
    else:
        # if multiple files were found, select the first one
        filename = file_list[0]
        print("surface data file: ", filename)
        dset = netCDF4.Dataset(filename, 'r+')
    
    # modify cover of specific PFTs
    dset['PCT_NAT_PFT'][0,:,:] = 100-plant_cover_obs # index 0: barren ground

    # modify land cover fractions (set whole gridcell to natural vegetation)
    dset['PCT_NATVEG'][:,:] = 100
    dset['PCT_CROP'][:,:] = 0
    dset['PCT_CFT'][:,:,:] = 0
    dset['PCT_WETLAND'][:,:] = 0
    dset['PCT_LAKE'][:,:] = 0
    dset['PCT_GLACIER'][:,:] = 0
    dset['PCT_URBAN'][:,:,:] = 0

    # Modify soil properties
    dset['ORGANIC'][0:3,:,:] = org_obs        # the layers of soil to modify depending on the availability of the data
    dset['PCT_SAND'][:,:,:] = sand_obs_mean
    dset['PCT_CLAY'][:,:,:] = clay_obs_mean
    dset['zbedrock'][:,:] = soil_depth_mean

    # Modify topography
    dset['SLOPE'][:,:] = slope_obs_mean

    dset.close()

-------------------------------------
site:  ALP1 Ulvehaugen
mean carbon content (%):  9.084
bulk density (g/cm^3):    0.2949073635697664
organic matter for model (kg/m3):  46.18859466668548
sand %:  19.72681304988417
clay %:  34.11610445379923
soil depth, m:  0.072865
slope:  14.45
surface data file:  C:\Users\evaler\OneDrive - Universitetet i Oslo\Eva\PHD\3_FATES_INCLINE\data_processed\surfacedata/ALP1\surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr2000_ALP1_c221011.nc


## Re-zip and upload the input data files

make zip archives

In [11]:
for i in range(0,1):
    print("making zipped folder: ", modified_surfacedata_path + "/" + siteID[i] + ".zip")
    shutil.make_archive(modified_surfacedata_path + "/" + siteID[i],
                        'zip', modified_surfacedata_path)
    print("zipping complete for ", siteID[i])
    

making zipped folder:  C:\Users\evaler\OneDrive - Universitetet i Oslo\Eva\PHD\3_FATES_INCLINE\data_processed\surfacedata/ALP1.zip


then change directories to local clone of repository, copy the files there, commit and push the changes

In [14]:
%%bash

cd /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/FATES_INCLINE
pwd

mv /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata/ALP1.zip data/
ls data/

/mnt/c/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE


In [ ]:
mv /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata/ALP2.zip data/
mv /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata/ALP3.zip data/
mv /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata/ALP4.zip data/

In [15]:
%%bash

cd /mnt/c/Users/evaler/OneDrive\ -\ Universitetet\ i\ Oslo/Eva/PHD/FATES_INCLINE
git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   README.md
	modified:   data/README.md
	modified:   src/DataPrep.R
	modified:   src/FATES_INCLINE.Rproj
	modified:   src/README.md
	modified:   src/SLA_notes
	modified:   src/biomass_incline.R
	modified:   src/change_param_file
	modified:   src/dataprep_surfacedata.ipynb
	modified:   src/ncdata.R
	modified:   src/output_plots
	modified:   src/plotting.py
	modified:   src/readingMultipleNC.R
	modified:   src/run_simulations_incline.ipynb
	modified:   src/settings_INCLINE
	modified:   src/traits_incline_dataprep.R
	modified:   src/traits_params.R
	modified:   src/workflow
	modified:   src/workflow_example.sh

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/ALP1.zip

no changes added to commi

In [ ]:
%%bash
git add .
git commit -m "add modified LSP input data"

In [ ]:
%%bash
git push